# Usage

In [1]:
import opendssdirect as dss

In [2]:
dss.__version__

'0.2.4'

### Import a file

Use the high level interface

In [3]:
dss.run_command('Redirect ./../../tests/data/13Bus/IEEE13Nodeckt.dss')

''

Use the low level interface

In [4]:
dss.dss_lib.DSSPut_Command('Redirect ./../../tests/data/13Bus/IEEE13Nodeckt.dss'.encode('ascii'))

b''

### Module `Loads`

The `dss` module has many submodules

In [5]:
import types

import inspect

for name, module in inspect.getmembers(dss):
    if isinstance(module, types.ModuleType) and not name.startswith('_'):
        print(f'dss.{name}')

dss.ActiveClass
dss.Basic
dss.Bus
dss.CapControls
dss.Capacitors
dss.Circuit
dss.CktElement
dss.Element
dss.Executive
dss.Fuses
dss.Generators
dss.Isource
dss.Lines
dss.LoadShape
dss.Loads
dss.Meters
dss.Monitors
dss.PDElements
dss.PVsystems
dss.Parser
dss.Properties
dss.Reclosers
dss.RegControls
dss.Relays
dss.Sensors
dss.Settings
dss.Solution
dss.SwtControls
dss.Topology
dss.Transformers
dss.Vsources
dss.XYCurves
dss.dss
dss.utils


Each submodule has functions that can be called.

In [6]:
for name, function in inspect.getmembers(dss.Loads):
    if callable(function) and not name.startswith('_'):
        print(f'dss.Loads.{name}')

dss.Loads.AllNames
dss.Loads.AllocationFactor
dss.Loads.CFactor
dss.Loads.CVRCurve
dss.Loads.CVRvars
dss.Loads.CVRwatts
dss.Loads.Class
dss.Loads.Count
dss.Loads.Daily
dss.Loads.Duty
dss.Loads.First
dss.Loads.Growth
dss.Loads.Idx
dss.Loads.IsDelta
dss.Loads.Model
dss.Loads.Name
dss.Loads.Next
dss.Loads.NumCust
dss.Loads.PF
dss.Loads.PctMean
dss.Loads.PctStdDev
dss.Loads.RelWeighting
dss.Loads.Rneut
dss.Loads.Spectrum
dss.Loads.Status
dss.Loads.Vmaxpu
dss.Loads.VminEmerg
dss.Loads.VminNorm
dss.Loads.Vminpu
dss.Loads.XfkVA
dss.Loads.Xneut
dss.Loads.Yearly
dss.Loads.ZipV
dss.Loads.kV
dss.Loads.kVABase
dss.Loads.kW
dss.Loads.kWh
dss.Loads.kWhDays
dss.Loads.kvar
dss.Loads.puSeriesRL


In [7]:
dss.Loads.AllNames()

['671',
 '634a',
 '634b',
 '634c',
 '645',
 '646',
 '692',
 '675a',
 '675b',
 '675c',
 '611',
 '652',
 '670a',
 '670b',
 '670c']

In [8]:
dss.Loads.Name()

'670c'

In [9]:
dss.Loads.kW()

117.0

To get all `kW`, one must iterate over all the loads

In [10]:
dss.Loads.First()

while True:
    
    print(
        'Name={name} \t kW={kW}'.format(
            name=dss.Loads.Name(), 
            kW=dss.Loads.kW()
        )
    )
    
    if not dss.Loads.Next() > 0:
        break

Name=671 	 kW=1155.0
Name=634a 	 kW=160.0
Name=634b 	 kW=120.0
Name=634c 	 kW=120.0
Name=645 	 kW=170.0
Name=646 	 kW=230.0
Name=692 	 kW=170.0
Name=675a 	 kW=485.0
Name=675b 	 kW=68.0
Name=675c 	 kW=290.0
Name=611 	 kW=170.0
Name=652 	 kW=128.0
Name=670a 	 kW=17.0
Name=670b 	 kW=66.0
Name=670c 	 kW=117.0


The `utils` module provides an `Iterator` class as a helper function

In [11]:
from opendssdirect.utils import Iterator

In [12]:
load_kW = [i() for i in Iterator(dss.Loads, 'kW')]

In [13]:
load_kW

[1155.0,
 160.0,
 120.0,
 120.0,
 170.0,
 230.0,
 170.0,
 485.0,
 68.0,
 290.0,
 170.0,
 128.0,
 17.0,
 66.0,
 117.0]

In [14]:
for i in Iterator(dss.Loads, 'Name'):
    
    print(
        'Name={name} \t kW={kW}'.format(
            name=i(), 
            kW=dss.Loads.kW()
        )
    )

Name=671 	 kW=1155.0
Name=634a 	 kW=160.0
Name=634b 	 kW=120.0
Name=634c 	 kW=120.0
Name=645 	 kW=170.0
Name=646 	 kW=230.0
Name=692 	 kW=170.0
Name=675a 	 kW=485.0
Name=675b 	 kW=68.0
Name=675c 	 kW=290.0
Name=611 	 kW=170.0
Name=652 	 kW=128.0
Name=670a 	 kW=17.0
Name=670b 	 kW=66.0
Name=670c 	 kW=117.0
